<a href="https://www.kaggle.com/code/archismancoder/t5-1-1-hf-seq2seq-trainer-batch4?scriptVersionId=236344825" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install transformers pandas openpyxl torch peft bs4 emoji joblib scikit-learn ray huggingface_hub datasets optuna lxml optim evaluate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
  Created wheel for optim: filename=optim-0.1.0-py2.py3-none-any.whl size=2707 sha256=0eafc3f0a31e6841cd361dbf17bdf64306cb6357db88d6afd76175460942683e
  Stored in directory: /root/.cache/pip/wheels/63/cd/16/e7762fdd7862a4f618fa7ca62119fac2112de90041cee77227
Successfully built optim


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-tachygraphy/Tachygraphy_EmotionMoodtags_Dataset.csv
/kaggle/input/dataset-tachygraphy/Tachygraphy_MicroText-AIO-V3.xlsx
/kaggle/input/dataset-tachygraphy/Tachygraphy_MicroText-AIO-Emotion_Mood_Tags-V3.xlsx
/kaggle/input/dataset-tachygraphy/Tachygraphy_dataset_main.csv
/kaggle/input/dataset-tachygraphy/Tachygraphy_MicroText-AIO-Sentiment_Polarities_3_Label-V3.xlsx
/kaggle/input/dataset-tachygraphy/Tachygraphy_MicroText-AIO-Sentiment_Polarities_2_Label-V3 (Neutral Omitted).xlsx
/kaggle/input/dataset-tachygraphy/Tachygraphy_MicroText-AIO-V2.xlsx


In [3]:
dataset = pd.read_excel('/kaggle/input/dataset-tachygraphy/Tachygraphy_MicroText-AIO-V3.xlsx')

In [4]:
df = dataset

In [5]:
df.rename(columns={'Informal Text':'input', 'Expanded Meaning':'target'}, inplace = True)

In [6]:
df.head()

,input,target
0,"omg, JEE prep is killing me rn","Oh my god, Joint Entrance Examination preparat..."
1,u up 4 a break b4 UPSC revision?,Are you up for a break before Union Public Ser...
2,"ttyl, finishing da CAT mock","Talk to you later, finishing the Common Admiss..."
3,"nah, dat GATE paper was brutal af","No, that Graduate Aptitude Test in Engineering..."
4,sup? u done w/ ur IIT assignment?,What's up? Are you done with your Indian Insti...


In [7]:
df['input'] = df['input'].astype(str)
df['target'] = df['target'].astype(str)

In [8]:
import torch
from torch.utils.data import DataLoader
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BertModel, BertTokenizer
# from torch.cuda.amp import autocast, GradScaler
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
import ray
from ray import tune
import torch.nn as nn

from torch.optim import AdamW
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel
from sklearn.model_selection import train_test_split
import optuna
# from optuna.integration import PyTorchLightningPruner
from ray import tune
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers import ASHAScheduler
# from ray.tune.integration.pytorch import TuneReportCallback
from torch.amp import GradScaler, autocast
# from ray.tune.integration.optuna import OptunaSearch
from ray.tune.search.optuna import OptunaSearch
from ray import tune
from ray.tune.search.hyperopt import HyperOptSearch
from torch import autocast
# from ray import tune
# from ray.tune.integration.tensorboard import TensorBoardReporter
from ray.tune.logger import TBXLogger
# from torch.utils.tensorboard import SummaryWriter
from ray.train import report
# from ray.tune.integration.jupyter import JupyterNotebookReporter
from ray.tune import JupyterNotebookReporter
# from torch.cuda.amp import GradScaler, autocast
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR
from ray.tune.schedulers import HyperBandScheduler, AsyncHyperBandScheduler


import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings


import emoji
from bs4 import BeautifulSoup
import os
import re
import string
import json


import argparse # CPMP

import scipy as sp

from tqdm.auto import tqdm

In [9]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}

def clean_text(text):
    '''Clean emoji, Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
#     text = emoji.demojize(text)
#     text = re.sub(r'\:(.*?)\:','',text)
#     text = str(text).lower()    #Making Text Lowercase
#     text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
#     text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, 0-9, "%", ".", "&", ",", "'", "?", "!", ",", "'", ";", "-")
    text = re.sub(r"[^a-zA-Z0-9?.!,¿'%&,';-]+", " ", text)
    return text

def clean_contractions(text, mapping):
    '''Clean contraction using contraction mapping'''    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    for word in mapping.keys():
        if ""+word+"" in text:
            text = text.replace(""+word+"", ""+mapping[word]+"")
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text

def clean_special_chars(text, punct, mapping):
    '''Cleans special characters present(if any)'''   
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

def correct_spelling(x, dic):
    '''Corrects common spelling errors'''   
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def remove_space(text):
    '''Removes awkward spaces'''   
    #Removes awkward spaces 
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
#     text = clean_contractions(text, contraction_mapping)
    text = clean_text(text)
#     text = clean_contractions(text, contraction_mapping)
#     text = clean_special_chars(text, punct, punct_mapping)
#     text = correct_spelling(text, mispell_dict)
    text = remove_space(text)
    return text

In [10]:
df['input'] = df['input'].apply(lambda x: text_preprocessing_pipeline(x))
df['target'] = df['target'].apply(lambda x: text_preprocessing_pipeline(x))

<ipython-input-9-93bb6d8e489f>:51: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'lxml').get_text()


In [11]:
df['input'] = df['input'].astype(str)
df['target'] = df['target'].astype(str)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
import torch
import transformers
from datasets import Dataset
from transformers import LlamaForCausalLM, LlamaTokenizer, TrainingArguments, Trainer, AutoTokenizer, BartForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import T5Tokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, T5ForConditionalGeneration

In [14]:
MODEL_NAME = 'google/t5-v1_1-base'

In [15]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [16]:
def test_train_split(dataset, test_ratio=0.1):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

In [17]:
train_ds_pd, validation_ds_pd = test_train_split(df)
print("{} examples in training, {} examples in testing.".format(
    len(train_ds_pd), len(validation_ds_pd)))

9314 examples in training, 966 examples in testing.


In [18]:
train_ds_pd = df

In [19]:
# def tokenize_function(batch):
#     model_inputs = tokenizer(batch["input"], padding="longest", truncation=False)
#     labels = tokenizer(batch["target"], padding="longest", truncation=False)
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

In [20]:
# def tokenize_function(batch):
#     model_inputs = tokenizer(batch["input"], padding="max_length", truncation=True, max_length=512)
#     labels = tokenizer(batch["target"], padding="max_length", truncation=True, max_length=512)

#     # Convert padding token (0) to -100 for loss calculation
#     labels["input_ids"] = [
#         [(l if l != tokenizer.pad_token_id else -100) for l in label]
#         for label in labels["input_ids"]
#     ]

#     # Debugging prints
#     print(f"Example input_ids: {model_inputs['input_ids'][0][:10]}")
#     print(f"Example labels: {labels['input_ids'][0][:10]}")
    
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

In [21]:
def tokenize_function(examples):
    model_inputs = tokenizer(examples["input"], padding="longest", truncation=True)
    labels = tokenizer(examples["target"], padding="longest", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [22]:
train_ds_pd = train_ds_pd.reset_index(drop=True)
validation_ds_pd = validation_ds_pd.reset_index(drop=True)

In [23]:
dataset1 = Dataset.from_pandas(train_ds_pd)
dataset2 = Dataset.from_pandas(validation_ds_pd)

In [24]:
tokenized_dataset = dataset1.map(tokenize_function, remove_columns=['input', 'target'], batched=True, batch_size=1)
tokenized_dataset2 = dataset2.map(tokenize_function, remove_columns=['input', 'target'], batched=True, batch_size=1)

Map:   0%|          | 0/10280 [00:00<?, ? examples/s]

Map:   0%|          | 0/966 [00:00<?, ? examples/s]

In [25]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,  # BF16 for H100
    device_map="auto"  # Auto GPU allocation
)

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [26]:
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding="longest")


In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
!pip install sacrebleu rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.4 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=4d6642cbabd86dcd6615efd934e5d2414bd9c36a841db8835f7b5b6cc2b7580d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [29]:
# import evaluate
# from sklearn.metrics.pairwise import cosine_similarity
# # from datasets import load_metric
# metric = evaluate.load("sacrebleu")

# # Load the sacrebleu metric
# bleu = evaluate.load("sacrebleu")
# # rouge = load_metric("rouge")
# rouge = evaluate.load("rouge")

# # Load metrics
# bleu = evaluate.load("sacrebleu")
# rouge = evaluate.load("rouge")

# def compute_metrics(eval_preds):
#     preds, labels = eval_preds

#     # No need for .cpu() since they're already NumPy arrays
#     preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # SacreBLEU expects references as a list of list of strings
#     bleu_labels = [[label] for label in decoded_labels]

#     # Compute BLEU score
#     result = bleu.compute(predictions=decoded_preds, references=bleu_labels)

#     # Move tensors to the same device
#     device = model.device

#     # Tokenize and convert decoded text to embeddings
#     pred_inputs = tokenizer(decoded_preds, return_tensors='pt', padding="longest", truncation=False).to(device)
#     # Fix here: decoded_labels should just be a flat list of strings
#     label_inputs = tokenizer(decoded_labels, return_tensors='pt', padding="longest", truncation=False).to(device)

#     pred_embeds = model.encoder(**pred_inputs).last_hidden_state.mean(dim=1)
#     label_embeds = model.encoder(**label_inputs).last_hidden_state.mean(dim=1)

#     # Compute cosine similarity
#     cosine_sim = cosine_similarity(pred_embeds.detach().cpu().numpy(), label_embeds.detach().cpu().numpy())
#     avg_cosine_sim = np.mean(np.diag(cosine_sim))

#     # Compute ROUGE-L score
#     rouge_result = rouge.compute(predictions=decoded_preds, references=bleu_labels, rouge_types=["rougeL"])
#     rouge_l_score = rouge_result["rougeL"].mid.fmeasure

#     # Log metrics
#     result = {
#         "bleu": result["score"],
#         "cosine_similarity": avg_cosine_sim,
#         "rougeL": rouge_l_score
#     }

#     return result

In [30]:
import evaluate
from sklearn.metrics.pairwise import cosine_similarity
# from datasets import load_metric
metric = evaluate.load("sacrebleu")

# Load the sacrebleu metric
bleu = evaluate.load("sacrebleu")
# rouge = load_metric("rouge")
rouge = evaluate.load("rouge")

# Load metrics
bleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # No need for .cpu() since they're already NumPy arrays
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # SacreBLEU expects references as a list of list of strings
    bleu_labels = [[label] for label in decoded_labels]

    # Compute BLEU score
    result = bleu.compute(predictions=decoded_preds, references=bleu_labels)

    # # Move tensors to the same device
    # device = model.device

    # # Tokenize and convert decoded text to embeddings
    # pred_inputs = tokenizer(decoded_preds, return_tensors='pt', padding="longest", truncation=False).to(device)
    # # Fix here: decoded_labels should just be a flat list of strings
    # label_inputs = tokenizer(decoded_labels, return_tensors='pt', padding="longest", truncation=False).to(device)

    # pred_embeds = model.encoder(**pred_inputs).last_hidden_state.mean(dim=1)
    # label_embeds = model.encoder(**label_inputs).last_hidden_state.mean(dim=1)

    # # Compute cosine similarity
    # cosine_sim = cosine_similarity(pred_embeds.detach().cpu().numpy(), label_embeds.detach().cpu().numpy())
    # avg_cosine_sim = np.mean(np.diag(cosine_sim))

    # Compute ROUGE-L score
    rouge_result = rouge.compute(predictions=decoded_preds, references=bleu_labels, rouge_types=["rougeL"])
    rouge_l_score = rouge_result["rougeL"]

    # Log metrics
    result = {
        "bleu": result["score"],
        # "cosine_similarity": avg_cosine_sim,
        "rougeL": rouge_l_score
    }

    return result

In [31]:
# from transformers import TrainingArguments

# training_args = TrainingArguments(
#     output_dir="./t5_1.1-checkpoints",
#     eval_strategy="steps",  # Evaluate at regular intervals
#     save_strategy="steps",  # Save model checkpoints at regular steps
#     logging_strategy="steps",  # Log at regular steps
#     logging_dir="/kaggle/working/logs",  # Directory for TensorBoard logs
#     logging_steps=50,  # Log loss/metrics every 50 steps
#     log_level="debug",
#     save_total_limit=2,  # Keep only the 2 latest checkpoints
#     per_device_train_batch_size=2,
#     per_device_eval_batch_size=2,
#     gradient_accumulation_steps=1,
#     learning_rate=5e-5,
#     weight_decay=0.01,
#     num_train_epochs=8,
#     lr_scheduler_type="linear",
#     warmup_steps=50,
#     fp16=True,  # Mixed precision for efficiency
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_loss",
#     report_to=["tensorboard"],
# )


In [32]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5_1.1-checkpoints",
    eval_strategy="steps",  # Evaluate at regular intervals
    save_strategy="steps",  # Save model checkpoints at regular steps
    save_steps=500,
    logging_strategy="steps",  # Log at regular steps
    logging_dir="/kaggle/working/logs",  # Directory for TensorBoard logs
    logging_steps=100,  # Log loss/metrics every 50 steps
    log_level="debug",
    save_total_limit=2,  # Keep only the 2 latest checkpoints
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=7,
    lr_scheduler_type="linear",
    warmup_steps=50,
    # fp16=True,  # Mixed precision for efficiency
    # bf16 = True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to=["tensorboard"],
    predict_with_generate=True,
    label_smoothing_factor=0.05,
)

In [33]:
print("Train Sample:", tokenized_dataset[0])
print("Val Sample:", tokenized_dataset2[0])


Train Sample: {'input_ids': [3, 32, 51, 122, 6, 446, 5080, 13422, 19, 9357, 140, 3, 52, 29, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [3359, 82, 8581, 6, 16761, 4443, 5219, 27275, 4537, 19, 10685, 53, 140, 269, 230, 5, 1]}
Val Sample: {'input_ids': [4752, 7, 3, 40, 51, 157, 3, 99, 3, 76, 3, 107, 208, 836, 3, 11973, 3358, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [863, 752, 140, 214, 3, 99, 25, 43, 8, 2557, 5623, 1799, 3358, 5, 1]}


In [34]:
# model.eval()
# inputs = tokenizer("i don8 no fr y hes soooo sad", return_tensors="pt").input_ids.to(model.device)
# output_ids = model.generate(inputs, max_length=256)
# print("Output:", tokenizer.decode(output_ids[0], skip_special_tokens=True))


In [35]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset,
#     eval_dataset=tokenized_dataset2,
#     tokenizer=tokenizer,
#     data_collator=data_collator,  # Ensures dynamic padding
#     compute_metrics=compute_metrics,
# )


# trainer.train()

In [36]:
print(model.generation_config)

GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0
}



In [37]:
# Get default GenerationConfig
default_gen_config = model.generation_config

# Override only max_length dynamically
default_gen_config.max_length = 1024  # Let it be dynamic
default_gen_config.max_new_tokens = 1024  # Optional, ensures dynamic generation

# Assign the updated config back to the model
model.generation_config = default_gen_config


In [38]:
print(model.generation_config)

GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "max_length": 1024,
  "max_new_tokens": 1024,
  "pad_token_id": 0
}



In [39]:
import warnings
warnings.filterwarnings("ignore")


In [40]:
import logging
logging.disable(logging.WARNING)


In [41]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset2,
    data_collator=data_collator,  # Ensures dynamic padding
    compute_metrics=compute_metrics,
    processing_class=tokenizer,
)


trainer.train()

Step,Training Loss,Validation Loss,Bleu,Rougel
100,15.940600,5.330214,2.761004,0.106524
200,8.902600,4.442748,2.497873,0.098735
300,6.532800,3.623720,5.553086,0.269273
400,4.634000,2.721606,11.117571,0.264779
500,3.876800,2.271857,0.000003,0.012106
600,3.374900,2.104081,26.013130,0.377566
700,2.976600,1.996019,34.522825,0.547640
800,2.788700,1.927153,43.705267,0.651087
900,2.652600,1.839053,51.616541,0.687171
1000,2.584200,1.787944,61.228162,0.716157


TrainOutput(global_step=17990, training_loss=1.693182230128256, metrics={'train_runtime': 26201.327, 'train_samples_per_second': 2.746, 'train_steps_per_second': 0.687, 'total_flos': 2596455439478784.0, 'train_loss': 1.693182230128256, 'epoch': 7.0})

In [42]:
# Save the fine-tuned model
model.save_pretrained("/kaggle/working/t5_1.1_finetuned")
tokenizer.save_pretrained("/kaggle/working/t5_1.1_finetuned")
print("Model saved successfully!")

Model saved successfully!


In [43]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("HF_READ_TOKEN")
# secret_value_1 = user_secrets.get_secret("HF_READ_WRITE_TOKEN")

In [44]:
from huggingface_hub import login

login(token=user_secrets.get_secret("HF_READ_TOKEN"))

In [45]:
from huggingface_hub import login

# Replace 'your_hf_token' with your actual Hugging Face API token
login(token=user_secrets.get_secret("HF_READ_WRITE_TOKEN"))

In [46]:
from huggingface_hub import HfApi, create_repo, upload_folder

# Define your Hugging Face repo ID
repo_id = "tachygraphy-microtrext-norm-org/T5-1.1-HF-seq2seq-Trainer-Batch4"  # Example

# Local directory where all your model files are stored
local_model_dir = "/kaggle/working/t5_1.1_finetuned"

# Initialize API instance
api = HfApi()

# Create the repo if it doesn't exist
create_repo(repo_id, exist_ok=True)

# Upload all the files from the local directory to the Hugging Face Hub
upload_folder(
    folder_path=local_model_dir,  # Path to the folder to upload
    repo_id=repo_id,              # Repo ID on Hugging Face
    repo_type="model"             # Specify it's a model repo
)

print(f"All files uploaded to Hugging Face under {repo_id}")

All files uploaded to Hugging Face under tachygraphy-microtrext-norm-org/T5-1.1-HF-seq2seq-Trainer-Batch4


In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [48]:
# def predict(model, tokenizer, text, device, num_return_sequences=1):
#     # Tokenize the input text with dynamic padding and length
#     padded = tokenizer(text, return_tensors='pt', truncation=False, padding="longest").to(device)

#     input_ids = padded['input_ids'].to(device)
#     attention_mask = padded['attention_mask'].to(device)

#     # Generate predictions
#     max_new_tokens = 1024
#     outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=max_new_tokens, num_return_sequences=num_return_sequences, do_sample=True, temperature=0.4)

#     # Decode predictions into human-readable text
#     predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)

#     return predictions



In [49]:
def predict(
    model, tokenizer, text, device, 
    num_return_sequences=1, 
    beams=None,  # Beam search
    do_sample=False,  # Sampling flag
    temp=None,  # Temperature (only for sampling)
    top_p=None, 
    top_k=None, 
    max_new_tokens=1024, 
    early_stopping=True
):
    # Tokenize input
    padded = tokenizer(text, return_tensors='pt', truncation=False, padding="longest").to(device)
    input_ids = padded['input_ids'].to(device)
    attention_mask = padded['attention_mask'].to(device)

    # Validate arguments
    if beams is not None and do_sample:
        raise ValueError("Cannot use `beams` and `do_sample=True` together. Choose either beam search (`beams=5`) or sampling (`do_sample=True, temp=0.7`).")
    
    if temp is not None and not do_sample:
        raise ValueError("`temp` (temperature) can only be used in sampling mode (`do_sample=True`).")

    if (top_p is not None or top_k is not None) and not do_sample:
        raise ValueError("`top_p` and `top_k` can only be used in sampling mode (`do_sample=True`).")

    # Beam search (Deterministic)
    if beams is not None:
        outputs = model.generate(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            max_new_tokens=max_new_tokens, 
            num_return_sequences=num_return_sequences, 
            num_beams=beams, 
            early_stopping=early_stopping, 
            do_sample=False  # No randomness
        )

    # Sampling Cases
    else:
        generate_args = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "max_new_tokens": max_new_tokens,
            "num_return_sequences": num_return_sequences,
            "do_sample": True,  # Enable stochastic sampling
            "temperature": temp if temp is not None else 0.7,  # Default temp if not passed
        }

        # Add `top_p` if set
        if top_p is not None:
            generate_args["top_p"] = top_p

        # Add `top_k` if set
        if top_k is not None:
            generate_args["top_k"] = top_k

        # Generate
        outputs = model.generate(**generate_args)

    # Decode predictions into human-readable text
    predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return predictions


In [50]:
# def predict(model, tokenizer, input_texts, device):
# #     model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
# #     model.load_state_dict(torch.load(model_path)['model_state_dict'])
#     model.to(device)
# #     model.eval()

#     predictions = []
#     with torch.no_grad():
# #         for text in input_texts:
#         text = input_texts
#         padded = tokenizer(text, return_tensors='pt', truncation=False, padding="longest").to(device)
        
#         input_ids = padded['input_ids'].to(device)
#         attention_mask = padded['attention_mask'].to(device)

#         # max_new_tokens = min(input_ids.shape[1] * 2, 1024)
#         max_new_tokens = 1024 # default since we are not sure about the word expansions, like brb -> be right back there can short form
#                               # of very large length words
            
#         preds = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=max_new_tokens, temperature=0.4, do_sample=True, top_k=50)
#         # preds = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=max_new_tokens)
#         pred_text = tokenizer.decode(preds[0], skip_special_tokens=True)
#         predictions.append(pred_text)
    
#     return predictions

In [51]:
# Test prediction on random input
random_input = "thx 4 the tip. will let u no. btw gotta use smpl things 4 now. tbh i'm tired, iykyk"
predicted_text = predict(model, tokenizer, random_input, device, num_return_sequences=3)
print(f"Input: {random_input}")
print(f"Predicted Meanings: {predicted_text} \n\n")

Input: thx 4 the tip. will let u no. btw gotta use smpl things 4 now. tbh i'm tired, iykyk
Predicted Meanings: ["Thanks for the tip. I will let you no. By the way I'm tired, amazed.", 'Thanks for the tip. Have to let you no. but by the way i am tired, I am tired,', "Thanks for the suggestion. gotta let you see a lot for now. To be honest I am tired, I'm tired, i am tired."] 




In [52]:
# Test prediction on random input
random_input = "pls hlp, idk wht 2 do"
predicted_text = predict(model, tokenizer, random_input, device, num_return_sequences=3)
print(f"Input: {random_input}")
print(f"Predicted Meanings: {predicted_text} \n\n")

Input: pls hlp, idk wht 2 do
Predicted Meanings: ['Please let me know what to do', "Please help, I don't know what to do.", 'Please hlp let me see what to do.'] 




In [53]:
# Test prediction on random input
random_input = "i don8 no fr y hes soooo sad"
predicted_text = predict(model, tokenizer, random_input, device, num_return_sequences=3)
print(f"Input: {random_input}")
print(f"Predicted Meanings: {predicted_text} \n\n")

Input: i don8 no fr y hes soooo sad
Predicted Meanings: ["i don't know for y hes s so sad", "I don't no for how they're sad", "I don't care for y hes soooo sad"] 




In [54]:
predict(model, tokenizer, "i don8 no fr y hes soooo sad", device, beams=5)


["I don't know no for y he's so sad"]

In [55]:
predict(model, tokenizer, "i don8 no fr y hes soooo sad", device, do_sample=True)


['abandonee no for y he is so sad']

In [56]:
predict(model, tokenizer, "i don8 no fr y hes soooo sad", device, do_sample=True, temp=0.4)


["i don't no for y he is so sad"]

In [57]:
predict(model, tokenizer, "i don8 no fr y hes soooo sad", device, do_sample=True, temp=0.4, top_p=0.9, top_k=50)


["I don't no for y he's so sad"]

In [58]:
!zip -r ./logs.zip /kaggle/working/logs



  adding: kaggle/working/logs/ (stored 0%)
  adding: kaggle/working/logs/events.out.tfevents.1745727773.c71b9cec9fcd.18.0 (deflated 69%)
